In [1]:
import requests
import json

In [2]:
# API call to iSeeOntoAPI to get the most similar cases
def get_cases_json_from_api(treeId_paremeter,usecaseId_parameter,topK_paremeter):
    """
        Function to get the solutions for that case in json format
    """
    url = "https://api-dev.isee4xai.com/api/trees/cbr_retrieve"

    payload = json.dumps({
      "treeId": treeId_paremeter,
      "usecaseId": usecaseId_parameter,
      "topk": topK_paremeter
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    #print(response.text)

    return json.loads(response.text)

In [3]:
# Build the node list from the behavior tree for the graph structure
def print_node_instances(node_id, nodes_dict, node_list, id_list): 
    node = nodes_dict[node_id]
    node_instance = node['Instance']
    if node_instance is None:
        return None
    node_list.append(node_instance)
    id_list.append(node_id)

    if 'firstChild' in node:
        first_child_id = node['firstChild']['Id']
        print_node_instances(first_child_id, nodes_dict, node_list, id_list)
        next_child = node['firstChild'].get('Next')

        while next_child is not None:
            next_child_id = next_child['Id']
            print_node_instances(next_child_id, nodes_dict, node_list, id_list)
            next_child = next_child.get('Next')

    return node_list, id_list

In [4]:
# Get the index of the parent in the behavior tree
def get_index(node_id, nodes_dict, id_list):
    node = nodes_dict[node_id]
    node_instance = node.get('Instance')
    node_index = id_list.index(node_id)
    node_index = node_index + 1

    return node_index, node_instance

In [5]:
# Find the parent of a node
def find_parent(node_id, node, parent_child_dict, id_list, nodes_dict):
    parent_index, parent_instance = get_index(node_id, nodes_dict, id_list)
    
    if 'firstChild' in node:
        first_child_id = node['firstChild']['Id']
        child_index, child_instance = get_index(first_child_id, nodes_dict, id_list)

        if parent_index not in parent_child_dict:
            parent_child_dict[parent_index] = []
        if child_index not in parent_child_dict[parent_index]:
            parent_child_dict[parent_index].append(child_index)
        
        next_child = node['firstChild'].get('Next')
        while next_child is not None:
            next_child_id = next_child['Id']
            child_index, child_instance = get_index(next_child_id, nodes_dict, id_list)
            if child_index not in parent_child_dict[parent_index]:
                parent_child_dict[parent_index].append(child_index)  # Add child index to the parent's list
            next_child = next_child.get('Next')

        return parent_instance

In [6]:
# Create a parent-child dictionary
def create_parent_child_dict(nodes_dict, node_list, id_list): 
    parent_child_dict = {}   
    # root = node_list[0] #r 
    parent_child_dict[0] = [1]  # Add root node with index 0

    for i, (instance, node_id) in enumerate(zip(node_list[1:], id_list), start=1):
        node_index = i
        node_id =id_list[node_index-1]
        node = nodes_dict[node_id]
        find_parent(node_id, node, parent_child_dict, id_list, nodes_dict)
    
    return parent_child_dict

In [7]:
# Build the adjacency list
def build_adjacency_list(node_list, parent_child_dict): 
    adjacency_list = [[] for _ in range(len(node_list))]

    for node_index, node_instance in enumerate(node_list):
        if node_index in parent_child_dict:
            children = parent_child_dict[node_index]
            adjacency_list[node_index] = children

    return adjacency_list

In [8]:
# function to translate the case solution to graph structure 
# This function must work for all the cases and the query 
# TODO
def translate_cases_from_json_to_graph(case):
  tree_dict, nodes_dict, parent_child_dict = {},{},{}
  node_list = ['r'] # Added 'r' as the default root node in the node list
  id_list =[] #List of node id's 
         

  for idx, obj in enumerate(case, start=1):
      trees = obj['data']['trees']
      # Get the 'nodes' from 'trees'
      for tree in trees:
          nodes = tree.get('nodes', {})
          nodes_dict.update(nodes)
          # Get the root node
          root_node_id = tree.get('root')    

      # Call the recursive function to print node instances
      node_list, id_list= print_node_instances(root_node_id, nodes_dict, node_list = ['r'], id_list =[])
      # Call the function to create the parent_child dictionary
      parent_child_dict = create_parent_child_dict(nodes_dict, node_list, id_list)
      # Build the adjacency list from the behavior tree
      adjacency_list = build_adjacency_list(node_list, parent_child_dict)
      
      tree_key = f'tree_{idx}'
    #   tree_dict[tree_key] = trees
      tree_dict[tree_key] = {
              'tree_json': trees,
              'tree_graph': {
                  'nodes': node_list,
                  'adj': adjacency_list
              }
      }

  return tree_dict


In [9]:
# Translate each BT in JSON format to graph structure with a list of nodes and adjacency.
treeId_parameter = "64b676baa737e466ce27f166" # query
usecaseId_parameter = "6426a68da3402ba28c44a7c0" # case of the query
topK_parameter = 5

cases_json = get_cases_json_from_api(treeId_parameter,usecaseId_parameter,topK_parameter)
tree_dict = translate_cases_from_json_to_graph(cases_json) 

print(tree_dict)

{'tree_1': {'tree_json': [{'version': '0.1.0', 'scope': 'tree', 'id': '33def3ec-31a8-47c1-856c-7fd724718df2', 'Instance': 'Explanation Experience', 'description': '', 'root': '546f5cee-68b0-4b90-85be-786b9957d03a', 'properties': {}, 'nodes': {'5112868d-f790-4665-ab3e-18a36a857363': {'id': '5112868d-f790-4665-ab3e-18a36a857363', 'Concept': 'Sequence', 'Instance': 'Sequence', 'description': '', 'properties': {}, 'display': {'x': -60, 'y': 168}, 'firstChild': {'Id': '85b9b22e-1b0a-4a9b-81a9-83952d27271a', 'Next': {'Id': '5829d6db-5011-4ad8-846a-ab8452c6be46', 'Next': None}}}, '546f5cee-68b0-4b90-85be-786b9957d03a': {'id': '546f5cee-68b0-4b90-85be-786b9957d03a', 'Concept': 'Priority', 'Instance': 'Priority', 'description': '', 'properties': {}, 'display': {'x': -60, 'y': 84}, 'firstChild': {'Id': '5112868d-f790-4665-ab3e-18a36a857363', 'Next': None}}, '85b9b22e-1b0a-4a9b-81a9-83952d27271a': {'id': '85b9b22e-1b0a-4a9b-81a9-83952d27271a', 'Concept': 'User Question', 'Instance': 'User Questio

In [10]:
# To save the json2graph output into a .txt file
# Specify the file path and name
file_path = "json2graphOutput.txt"

# Open the file in write mode
with open(file_path, "w") as file:
    # Write the dictionary as a JSON string
    json.dump(tree_dict, file, indent=4)